# Chapter 12: Sequence-to-sequence Learning (Part 2) – Attention Mechanism

Bab ini berfokus pada peningkatan model Seq2Seq menggunakan **Bahdanau Attention** (sering disebut *Additive Attention*). Mekanisme ini memungkinkan Decoder untuk fokus pada bagian tertentu dari kalimat sumber saat menghasilkan setiap kata terjemahan.

---

## 12.1 Mengapa Butuh Attention? (Model Improvement)
Pada Seq2Seq standar, seluruh informasi kalimat sumber dipaksa masuk ke dalam satu *context vector* statis. Ini menciptakan *bottleneck* untuk kalimat yang panjang.
* **Bahdanau Attention**: Menghasilkan representasi dinamis yang kaya dengan melihat kembali seluruh *hidden states* dari Encoder (masa lalu dan masa depan sekuens).
* **Mekanisme**: Output dari layer Attention digabungkan (*concatenate*) dengan output GRU pada Decoder. Informasi gabungan ini kemudian diteruskan ke lapisan *dense* untuk memprediksi kata berikutnya.



---

## 12.2 Implementasi Bahdanau Attention
Karena mekanisme ini melibatkan logika kustom dalam perhitungan skor bobot, kita sering kali harus membangunnya secara manual.
* **Keras Subclassing API**: Digunakan untuk membuat layer `Attention` kustom. Kita mendefinisikan bobot $W_1$, $W_2$, dan $V$ untuk menghitung skor keselarasan antara *hidden state* Decoder dan output Encoder.
* **Tips Komputasi**: Menyamakan jumlah unit RNN pada Encoder dan Decoder sangat disarankan untuk menyederhanakan operasi perkalian matriks di dalam mekanisme attention.

---

## 12.3 Alur Membangun Model Akhir
Membangun model Seq2Seq dengan Attention memerlukan penggabungan beberapa komponen fungsional:
1.  **Vectorizer**: Menggunakan `get_vectorizer()` untuk standarisasi teks.
2.  **Encoder**: Menggunakan `get_encoder()` untuk mengekstraksi fitur sekuensial.
3.  **Final Model**: Menggunakan `get_final_seq2seq_model_with_attention()` untuk merakit Encoder, Decoder, dan Layer Attention menjadi satu kesatuan.
4.  **Training**: Model dilatih menggunakan fungsi pembantu `train_model()` dengan menyuplai data yang sudah dibagi (`train_df`, `valid_df`, `test_df`) serta menentukan `batch_size`.

---

## 12.4 Visualizing the Attention (Heatmaps)
Salah satu keunggulan luar biasa dari mekanisme Attention adalah **Interpretabilitas** (model bisa menjelaskan apa yang dia "pikirkan").
* **Attention Visualizer**: Fungsi ini memuat model yang telah disimpan dan mengekstraksi bobot perhatian (*attention weights*) untuk setiap kata yang dihasilkan.
* **Heatmap Matplotlib**: Menggunakan `matplotlib` untuk menampilkan matriks hubungan antara kata sumber (sumbu X) dan kata target (sumbu Y).
* **Interpretasi Warna**: Warna yang lebih terang atau piksel yang lebih pekat menunjukkan bahwa model memberikan "perhatian" yang lebih besar pada kata sumber tersebut saat menerjemahkan kata target tertentu.



---

## 12.5 Tip Implementasi di Colab
Saat menggunakan Subclassing untuk Attention, pastikan kamu menghitung skor dengan benar sebelum menerapkan Softmax:

```python
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        # query (decoder hidden state) shape: (batch_size, hidden size)
        # values (encoder outputs) shape: (batch_size, max_len, hidden size)
        
        # Menghitung skor perhatian
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(tf.expand_dims(query, 1))))
        
        # attention_weights shape: (batch_size, max_len, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
        
        # Menghitung context_vector
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights